# Live coding stuff

In [1]:
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip

--2021-04-13 14:10:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘smsspamcollection.zip’ not modified on server. Omitting download.



In [2]:
!unzip -n smsspamcollection.zip
!ls

Archive:  smsspamcollection.zip
replace SMSSpamCollection? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace SMSSpamCollection? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: SMSSpamCollection       
  inflating: readme                  
readme	sample_data  SMSSpamCollection	smsspamcollection.zip


In [3]:
# Read raw data
with open('SMSSpamCollection', 'r') as rawdata:
  textlines = rawdata.readlines()

# Preprocessing
textlines = [line.strip() for line in textlines]

# Create data containers
X_raw = list()
y_raw = list()

# Split the posts
for textline in textlines:
  label, text = textline.split('\t')
  X_raw.append(text)
  y_raw.append(label)

assert len(set(y_raw)) == 2

In [4]:
from sklearn.model_selection import train_test_split

X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X_raw, y_raw, test_size=0.2, random_state=42)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_raw_train)

voc = set(vectorizer.get_feature_names())
assert 'santa' in voc
print("Size of the vocabulary %i" % len(voc))

Size of the vocabulary 7726


In [6]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(y_raw)
print("Classes:", label_encoder.classes_)

y = label_encoder.transform(y_raw_train)

y_raw_train[:10], y[:10]

Classes: ['ham' 'spam']


(['spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham'],
 array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0]))

In [13]:
from sklearn.neighbors import NearestCentroid
classifier = NearestCentroid()
classifier.fit(X, y)

print("Training accuracy: %.1f%%" % (100*classifier.score(X, y)))

Training accuracy: 97.5%


In [15]:
label_encoder.inverse_transform(classifier.predict(X))

array(['spam', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [16]:
X_test = vectorizer.transform(X_raw_test)
y_test = label_encoder.transform(y_raw_test)

print("Test accuracy: %.1f%%" % (100*classifier.score(X_test, y_test)))

Test accuracy: 97.1%


In [21]:
import numpy as np
majority_baseline = np.sum(y_test==0)/len(y_test)
majority_baseline

0.8556053811659193